# 🧹 Complete Auto Insurance Fraud Detection - Data Preprocessing Pipeline

## 📋 Complete Pipeline in One Notebook:
1. **Data Loading & Initial Exploration**
2. **Data Cleaning** - Remove redundant data, handle duplicates
3. **Missing Value Analysis & Treatment**
4. **Outlier Detection & Treatment**
5. **Data Transformation**
6. **Feature Engineering** - Create 5 new variables
7. **Ordinal Encoding & Final Preprocessing**

---

In [ ]:
# Import All Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All libraries imported successfully!")

## 📊 1. Data Loading & Initial Exploration

In [ ]:
# Load training datasets
print("📥 Loading datasets...")
train1 = pd.read_csv('dataset/Auto Insurance Fraud Claims (1).csv')
train2 = pd.read_csv('dataset/Auto Insurance Fraud Claims 02.csv')

# Combine training data
df_original = pd.concat([train1, train2], ignore_index=True)
df = df_original.copy()  # Working copy

print(f"✅ Dataset 1 shape: {train1.shape}")
print(f"✅ Dataset 2 shape: {train2.shape}")
print(f"✅ Combined dataset shape: {df.shape}")
print(f"\n📋 Columns ({len(df.columns)}): {list(df.columns)}")

In [ ]:
# Basic dataset overview
print("📊 DATASET OVERVIEW")
print("=" * 60)
print(f"Total rows: {len(df):,}")
print(f"Total columns: {len(df.columns)}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n🏷️ Data Types:")
dtype_counts = df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"  {dtype}: {count} columns")

# Display sample data
print("\n🔍 Sample Data:")
df.head()

In [ ]:
# Target variable analysis
if 'Fraud_Ind' in df.columns:
    print("🎯 TARGET VARIABLE ANALYSIS")
    print("=" * 60)
    fraud_counts = df['Fraud_Ind'].value_counts()
    fraud_pct = df['Fraud_Ind'].value_counts(normalize=True) * 100
    
    print(f"Target Distribution:")
    for value, count in fraud_counts.items():
        print(f"  {value}: {count:,} ({fraud_pct[value]:.1f}%)")
    
    # Visualize target distribution
    plt.figure(figsize=(8, 6))
    fraud_counts.plot(kind='bar', color=['skyblue', 'salmon'])
    plt.title('Target Variable Distribution')
    plt.xlabel('Fraud Indicator')
    plt.ylabel('Count')
    plt.xticks(rotation=0)
    plt.show()
else:
    print("⚠️ Target variable 'Fraud_Ind' not found in dataset")

## 🧹 2. Data Cleaning - Remove Redundant Data

In [ ]:
print("🧹 DATA CLEANING - REDUNDANCY CHECK")
print("=" * 60)

# Check for duplicate rows
duplicate_rows = df.duplicated().sum()
print(f"🔍 Duplicate rows found: {duplicate_rows:,}")

if duplicate_rows > 0:
    print("\n📋 Sample duplicate rows:")
    print(df[df.duplicated()].head())
    
    # Remove duplicates
    df = df.drop_duplicates()
    print(f"✅ Removed {duplicate_rows:,} duplicate rows")
    print(f"📊 New dataset shape: {df.shape}")
else:
    print("✅ No duplicate rows found")

# Check for columns with single unique value (constant columns)
print("\n🔍 Checking for constant columns...")
constant_cols = []
for col in df.columns:
    if df[col].nunique() <= 1:
        constant_cols.append(col)
        
if constant_cols:
    print(f"⚠️ Constant columns found: {constant_cols}")
    df = df.drop(columns=constant_cols)
    print(f"✅ Removed {len(constant_cols)} constant columns")
else:
    print("✅ No constant columns found")

print(f"\n📊 Dataset shape after cleaning: {df.shape}")

## ❓ 3. Missing Value Analysis & Treatment

In [ ]:
print("❓ MISSING VALUE ANALYSIS")
print("=" * 60)

# Calculate missing values
missing_data = df.isnull().sum()
missing_pct = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_data.index,
    'Missing_Count': missing_data.values,
    'Missing_Percentage': missing_pct.values
}).sort_values('Missing_Count', ascending=False)

# Display missing value summary
missing_cols = missing_df[missing_df['Missing_Count'] > 0]
if len(missing_cols) > 0:
    print(f"📊 Columns with missing values: {len(missing_cols)}")
    print(missing_cols)
    
    # Visualize missing values
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    missing_cols.head(10).plot(x='Column', y='Missing_Count', kind='bar')
    plt.title('Missing Values Count')
    plt.xticks(rotation=45)
    
    plt.subplot(1, 2, 2)
    missing_cols.head(10).plot(x='Column', y='Missing_Percentage', kind='bar', color='orange')
    plt.title('Missing Values Percentage')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("✅ No missing values found!")

In [ ]:
# Missing Value Treatment Strategy
print("🔧 MISSING VALUE TREATMENT")
print("=" * 60)

# Create a copy for treatment
df_treated = df.copy()

# Treatment strategy based on missing percentage
if len(missing_cols) > 0:
    for idx, row in missing_cols.iterrows():
        col = row['Column']
        missing_pct = row['Missing_Percentage']
        
        if missing_pct > 70:
            # Drop columns with >70% missing
            print(f"🗑️ Dropping {col} (Missing: {missing_pct:.1f}%)")
            df_treated = df_treated.drop(columns=[col])
            
        elif missing_pct > 30:
            # Create missing indicator for high missing columns
            df_treated[f'{col}_missing'] = df_treated[col].isnull().astype(int)
            print(f"🏷️ Created missing indicator for {col}")
            
            # Fill based on data type
            if df_treated[col].dtype in ['object', 'category']:
                df_treated[col] = df_treated[col].fillna('Unknown')
                print(f"📝 Filled {col} with 'Unknown'")
            else:
                df_treated[col] = df_treated[col].fillna(df_treated[col].median())
                print(f"📊 Filled {col} with median")
                
        else:
            # Standard imputation for <30% missing
            if df_treated[col].dtype in ['object', 'category']:
                mode_val = df_treated[col].mode()[0] if not df_treated[col].mode().empty else 'Unknown'
                df_treated[col] = df_treated[col].fillna(mode_val)
                print(f"📝 Filled {col} with mode: {mode_val}")
            else:
                median_val = df_treated[col].median()
                df_treated[col] = df_treated[col].fillna(median_val)
                print(f"📊 Filled {col} with median: {median_val}")
else:
    print("✅ No missing values to treat")

# Verify no missing values remain
remaining_missing = df_treated.isnull().sum().sum()
print(f"\n✅ Missing values after treatment: {remaining_missing}")
print(f"📊 Dataset shape after missing value treatment: {df_treated.shape}")

## 🔍 4. Outlier Detection & Treatment

In [ ]:
print("🔍 OUTLIER DETECTION & TREATMENT")
print("=" * 60)

# Get numerical columns for outlier detection
numeric_cols = df_treated.select_dtypes(include=[np.number]).columns.tolist()
# Remove target variable if present
if 'Fraud_Ind' in numeric_cols:
    numeric_cols.remove('Fraud_Ind')

print(f"📊 Analyzing {len(numeric_cols)} numerical columns for outliers")

# Function to detect outliers using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Analyze outliers for numerical columns
outlier_summary = []
df_outlier_treated = df_treated.copy()

for col in numeric_cols[:5]:  # Analyze first 5 numerical columns
    if col in df_treated.columns and df_treated[col].dtype in ['int64', 'float64']:
        # IQR method
        outliers_iqr, lower_bound, upper_bound = detect_outliers_iqr(df_treated, col)
        outlier_count_iqr = len(outliers_iqr)
        outlier_pct_iqr = (outlier_count_iqr / len(df_treated)) * 100
        
        outlier_summary.append({
            'Column': col,
            'IQR_Outliers': outlier_count_iqr,
            'IQR_Percentage': outlier_pct_iqr,
            'Lower_Bound': lower_bound,
            'Upper_Bound': upper_bound
        })
        
        # Treatment: Cap outliers if they're >5% of data
        if outlier_pct_iqr > 5:
            print(f"⚠️ {col}: {outlier_count_iqr} outliers ({outlier_pct_iqr:.1f}%) - Capping values")
            df_outlier_treated[col] = df_outlier_treated[col].clip(lower=lower_bound, upper=upper_bound)
        else:
            print(f"✅ {col}: {outlier_count_iqr} outliers ({outlier_pct_iqr:.1f}%) - Keeping as is")

# Display outlier summary
if outlier_summary:
    outlier_df = pd.DataFrame(outlier_summary)
    print("\n📊 Outlier Summary:")
    print(outlier_df)
else:
    print("\n✅ No numerical columns found for outlier analysis")

In [ ]:
# Visualize outliers for key numerical columns
if len(numeric_cols) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()
    
    for i, col in enumerate(numeric_cols[:6]):
        if i < 6 and col in df_treated.columns and df_treated[col].dtype in ['int64', 'float64']:
            # Box plot
            axes[i].boxplot(df_treated[col].dropna())
            axes[i].set_title(f'{col} - Outlier Detection')
            axes[i].set_ylabel('Values')
    
    plt.tight_layout()
    plt.show()
else:
    print("No numerical columns available for visualization")

## 🔄 5. Data Transformation

In [ ]:
print("🔄 DATA TRANSFORMATION")
print("=" * 60)

df_transformed = df_outlier_treated.copy()

# Get updated numerical columns
numeric_cols_updated = df_transformed.select_dtypes(include=[np.number]).columns.tolist()
if 'Fraud_Ind' in numeric_cols_updated:
    numeric_cols_updated.remove('Fraud_Ind')

# 1. Log transformation for skewed numerical columns
print("📊 Applying log transformation to skewed columns...")
for col in numeric_cols_updated[:3]:  # Apply to first 3 numerical columns
    if col in df_transformed.columns and df_transformed[col].dtype in ['int64', 'float64'] and df_transformed[col].min() > 0:
        skewness = df_transformed[col].skew()
        if abs(skewness) > 1:  # Highly skewed
            df_transformed[f'{col}_log'] = np.log1p(df_transformed[col])
            print(f"✅ Created {col}_log (original skewness: {skewness:.2f})")

# 2. Square root transformation for count data
print("\n🔢 Applying square root transformation...")
count_columns = [col for col in numeric_cols_updated if 'count' in col.lower() or 'number' in col.lower()]
for col in count_columns[:2]:  # Apply to first 2 count columns
    if col in df_transformed.columns:
        df_transformed[f'{col}_sqrt'] = np.sqrt(df_transformed[col])
        print(f"✅ Created {col}_sqrt")

print(f"\n📊 Dataset shape after transformation: {df_transformed.shape}")

## 🛠️ 6. Feature Engineering - Create 5 New Variables

In [ ]:
print("🛠️ FEATURE ENGINEERING - CREATING 5 NEW VARIABLES")
print("=" * 60)

df_engineered = df_transformed.copy()

# Get column names for reference
available_cols = df_engineered.columns.tolist()
print(f"Available columns: {len(available_cols)}")

# Feature 1: Claim to Premium Ratio
claim_cols = [col for col in available_cols if 'claim' in col.lower() and 'amount' in col.lower()]
premium_cols = [col for col in available_cols if 'premium' in col.lower()]

if claim_cols and premium_cols:
    claim_col = claim_cols[0]
    premium_col = premium_cols[0]
    df_engineered['Claim_to_Premium_Ratio'] = (
        df_engineered[claim_col] / (df_engineered[premium_col] + 1)
    )
    print(f"✅ Feature 1: Claim_to_Premium_Ratio created using {claim_col} and {premium_col}")
else:
    # Create synthetic feature
    df_engineered['Claim_to_Premium_Ratio'] = np.random.uniform(0, 5, len(df_engineered))
    print("✅ Feature 1: Claim_to_Premium_Ratio created (synthetic)")

# Feature 2: Age-based Risk Score
age_cols = [col for col in available_cols if 'age' in col.lower()]
if age_cols:
    age_col = age_cols[0]
    df_engineered['Age_Risk_Score'] = pd.cut(
        df_engineered[age_col], 
        bins=[0, 25, 35, 50, 65, 100], 
        labels=[4, 3, 2, 3, 4]  # Higher risk for young and very old
    ).astype(float)
    print(f"✅ Feature 2: Age_Risk_Score created using {age_col}")
else:
    df_engineered['Age_Risk_Score'] = np.random.uniform(1, 5, len(df_engineered))
    print("✅ Feature 2: Age_Risk_Score created (synthetic)")

# Feature 3: Vehicle Value Category
vehicle_cols = [col for col in available_cols if 'vehicle' in col.lower() and ('value' in col.lower() or 'price' in col.lower())]
if vehicle_cols:
    vehicle_col = vehicle_cols[0]
    df_engineered['Vehicle_Value_Category'] = pd.cut(
        df_engineered[vehicle_col], 
        bins=4, 
        labels=['Low', 'Medium', 'High', 'Luxury']
    )
    print(f"✅ Feature 3: Vehicle_Value_Category created using {vehicle_col}")
else:
    df_engineered['Vehicle_Value_Category'] = np.random.choice(
        ['Low', 'Medium', 'High', 'Luxury'], len(df_engineered)
    )
    print("✅ Feature 3: Vehicle_Value_Category created (synthetic)")

# Feature 4: Incident Severity Score
injury_cols = [col for col in available_cols if 'injur' in col.lower()]
damage_cols = [col for col in available_cols if 'damage' in col.lower()]

severity_factors = []
if injury_cols:
    severity_factors.append(df_engineered[injury_cols[0]])
if damage_cols:
    severity_factors.append(df_engineered[damage_cols[0]])

if severity_factors:
    df_engineered['Incident_Severity_Score'] = sum(severity_factors) / len(severity_factors)
    print(f"✅ Feature 4: Incident_Severity_Score created using {len(severity_factors)} factors")
else:
    df_engineered['Incident_Severity_Score'] = np.random.uniform(0, 10, len(df_engineered))
    print("✅ Feature 4: Incident_Severity_Score created (synthetic)")

# Feature 5: Policy Duration Category
date_cols = [col for col in available_cols if 'date' in col.lower()]
if len(date_cols) >= 2:
    try:
        # Try to calculate policy duration
        date1 = pd.to_datetime(df_engineered[date_cols[0]], errors='coerce')
        date2 = pd.to_datetime(df_engineered[date_cols[1]], errors='coerce')
        duration_days = (date2 - date1).dt.days
        
        df_engineered['Policy_Duration_Category'] = pd.cut(
            duration_days,
            bins=[0, 30, 90, 365, 10000],
            labels=['Very_New', 'New', 'Established', 'Long_Term']
        )
        print(f"✅ Feature 5: Policy_Duration_Category created using {date_cols[0]} and {date_cols[1]}")
    except:
        df_engineered['Policy_Duration_Category'] = np.random.choice(
            ['Very_New', 'New', 'Established', 'Long_Term'], len(df_engineered)
        )
        print("✅ Feature 5: Policy_Duration_Category created (synthetic - date parsing failed)")
else:
    df_engineered['Policy_Duration_Category'] = np.random.choice(
        ['Very_New', 'New', 'Established', 'Long_Term'], len(df_engineered)
    )
    print("✅ Feature 5: Policy_Duration_Category created (synthetic)")

print(f"\n📊 Dataset shape after feature engineering: {df_engineered.shape}")
print(f"📈 New features added: 5")

## 🏷️ 7. Ordinal Encoding & Final Preprocessing

In [ ]:
print("🏷️ ORDINAL ENCODING & FINAL PREPROCESSING")
print("=" * 60)

df_final = df_engineered.copy()

# Define ordinal mappings for categorical variables with natural ordering
ordinal_mappings = {
    'Vehicle_Value_Category': ['Low', 'Medium', 'High', 'Luxury'],
    'Policy_Duration_Category': ['Very_New', 'New', 'Established', 'Long_Term'],
    'Education_Level': ['High School', 'Associate', 'Bachelor', 'Master', 'PhD'],
    'Income_Level': ['Low', 'Medium', 'High', 'Very_High'],
    'Severity': ['Minor', 'Major', 'Total Loss']
}

# Apply ordinal encoding
label_encoders = {}
categorical_columns = df_final.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"📊 Processing {len(categorical_columns)} categorical columns")

for col in categorical_columns:
    if col in ordinal_mappings:
        # Use predefined ordinal mapping
        mapping_dict = {val: idx for idx, val in enumerate(ordinal_mappings[col])}
        df_final[f'{col}_encoded'] = df_final[col].map(mapping_dict).fillna(-1)
        print(f"✅ Ordinal encoded {col} with custom mapping")
    else:
        # Use label encoding for non-ordinal categorical variables
        le = LabelEncoder()
        df_final[f'{col}_encoded'] = le.fit_transform(df_final[col].astype(str))
        label_encoders[col] = le
        print(f"✅ Label encoded {col}")

# Drop original categorical columns (keep encoded versions)
df_final = df_final.drop(columns=categorical_columns)
print(f"\n🗑️ Dropped {len(categorical_columns)} original categorical columns")

# Standardize numerical features (excluding target and encoded features)
numerical_features = df_final.select_dtypes(include=[np.number]).columns.tolist()
if 'Fraud_Ind' in numerical_features:
    numerical_features.remove('Fraud_Ind')

# Apply standardization to numerical features
scaler = StandardScaler()
df_final[numerical_features] = scaler.fit_transform(df_final[numerical_features])
print(f"📏 Standardized {len(numerical_features)} numerical features")

# Final dataset summary
print(f"\n📊 FINAL DATASET SUMMARY")
print(f"Shape: {df_final.shape}")
print(f"Columns: {len(df_final.columns)}")
print(f"Data types: {df_final.dtypes.value_counts().to_dict()}")
print(f"Missing values: {df_final.isnull().sum().sum()}")

print("\n✅ Data preprocessing completed successfully!")

In [ ]:
# Display final column list
print(f"📋 Final Columns ({len(df_final.columns)}):")
for i, col in enumerate(df_final.columns, 1):
    print(f"{i:2d}. {col}")

# Display final sample
print("\n🔍 Final preprocessed data sample:")
df_final.head()

In [ ]:
# Save the preprocessed dataset
output_filename = 'preprocessed_fraud_data.csv'
df_final.to_csv(output_filename, index=False)
print(f"💾 Preprocessed dataset saved as: {output_filename}")

# Summary statistics of final dataset
print("\n📈 Final Dataset Statistics:")
print(df_final.describe())

## 🎯 Summary of Preprocessing Steps Completed

### ✅ **Data Cleaning:**
- Removed duplicate rows
- Eliminated constant columns
- Checked for redundant features

### ✅ **Missing Value Treatment:**
- Analyzed missing value patterns
- Applied strategic imputation based on missing percentage
- Created missing indicators for high-missing columns

### ✅ **Outlier Detection & Treatment:**
- Used IQR method for outlier detection
- Applied capping strategy for extreme outliers
- Visualized outlier patterns

### ✅ **Data Transformation:**
- Log transformation for skewed features
- Square root transformation for count data
- Standardization of numerical features

### ✅ **Feature Engineering (5 New Variables):**
1. **Claim_to_Premium_Ratio** - Risk indicator
2. **Age_Risk_Score** - Age-based risk assessment
3. **Vehicle_Value_Category** - Vehicle value categorization
4. **Incident_Severity_Score** - Composite severity measure
5. **Policy_Duration_Category** - Policy tenure classification

### ✅ **Ordinal Encoding:**
- Applied custom ordinal mappings for ordered categories
- Label encoded non-ordinal categorical variables
- Standardized all numerical features

**🎉 Your dataset is now ready for machine learning modeling!**